# Data Cleaning and Normalization for Gene Expression Analysis

This notebook outlines the steps for cleaning and normalizing a gene expression dataset. The key steps are:

1. Data Reading:
Reading the data as a dataframe in Pandas

2. Column Renaming:
This is a step done for better readability and easier data manipulation. I rename the columns to make them more understandable.

3. String Splitting:
Here, I split the Gene_ID into two separate identifiers: ENSG and HGNC. This is also a data manipulation step and is done to make the data easier to work with.

4. Data Normalization:
Normalization is performed to make the data comparable across different samples. I use Counts Per Million (CPM) normalization, which is commonly used in RNA-seq data analysis. This involves dividing each raw value by the sum of all values in its column and then multiplying by 1,000,000.

5. Data Filtering:
After normalization, I filter out genes that have a mean expression value across all samples that is less than 1. This is done to remove genes that are lowly expressed across all samples, as they are less likely to be of interest in the following analyses.

In [29]:
import pandas as pd

In [48]:
# Step 1: Reading the Data
# Read the raw matrix into a DataFrame
df = pd.read_csv('../data/GSE162285_gene_raw_counts_matrix.txt', delimiter='\t')
print("Initial Data:\n")
print(df.head(2))

Initial Data:

          ENSG|HGNC_symbol  \
0  ENSG00000223972|DDX11L1   
1   ENSG00000227232|WASH7P   

   20170417_MDAMB231Org_Veh1_ED3996-2_S1_R1_001.trimmed.fastq.gz.ReadsPerGene.out.tab  \
0                                                  0                                    
1                                                  1                                    

   20170417_MDAMB231Org_Veh2_ED3996-2_S2_R1_001.trimmed.fastq.gz.ReadsPerGene.out.tab  \
0                                                  0                                    
1                                                  2                                    

   20170417_MDAMB231Org_Veh3_ED3996-2_S3_R1_001.trimmed.fastq.gz.ReadsPerGene.out.tab  \
0                                                  0                                    
1                                                  0                                    

   20170417_MDAMB231Org_Doc1_ED3996-2_S10_R1_001.trimmed.fastq.gz.ReadsPerGene.out.tab  \


In [50]:
# Step 2: Renaming Columns
# Rename the columns for better readability
df.columns = ['Gene_ID'] + [f'Sample_{i}' for i in range(1, df.shape[1])]
print("\nColumns after renaming:\n")
#print(df.columns)
print(df.head())


Columns after renaming:

                     Gene_ID  Sample_1  Sample_2  Sample_3  Sample_4  \
0    ENSG00000223972|DDX11L1         0         0         0         3   
1     ENSG00000227232|WASH7P         1         2         0         2   
2  ENSG00000278267|MIR6859-1         1         0         1         0   
3           ENSG00000243485|         0         0         0         3   
4  ENSG00000274890|MIR1302-2         0         0         0         0   

   Sample_5  Sample_6  Sample_7  Sample_8  Sample_9  ...  Sample_64  \
0         0         1         0         0         0  ...          0   
1         1         0         1         0         0  ...          0   
2         0         0         0         0         0  ...          1   
3         1         0         0         0         0  ...          0   
4         0         0         0         0         0  ...          0   

   Sample_65  Sample_66  Sample_67  Sample_68  Sample_69  Sample_70  \
0          1          0          0         

In [68]:
# Step 3: Splitting Gene_ID
# Split the 'Gene_ID' into 'ENSG' and 'HGNC' based on the delimiter '|'
df[['ENSG', 'HGNC']] = df['Gene_ID'].str.split('|', expand=True)
print("Data after splitting Gene_ID:\n")
print(df.head())

Data after splitting Gene_ID:

                     Gene_ID  Sample_1  Sample_2  Sample_3  Sample_4  \
0    ENSG00000223972|DDX11L1         0         0         0         3   
1     ENSG00000227232|WASH7P         1         2         0         2   
2  ENSG00000278267|MIR6859-1         1         0         1         0   
3           ENSG00000243485|         0         0         0         3   
4  ENSG00000274890|MIR1302-2         0         0         0         0   

   Sample_5  Sample_6  Sample_7  Sample_8  Sample_9  ...  Sample_66  \
0         0         1         0         0         0  ...          0   
1         1         0         1         0         0  ...          4   
2         0         0         0         0         0  ...          1   
3         1         0         0         0         0  ...          0   
4         0         0         0         0         0  ...          0   

   Sample_67  Sample_68  Sample_69  Sample_70  Sample_71  Sample_72  \
0          0          0          0    

In [85]:
# Step 4: Normalizing the Data
# Normalize the data by dividing each value by the sum of its column and then multiplying by 1e6
# Include the 'Gene_ID', 'ENSG', and 'HGNC' columns in the normalization process
df_normalized = df.iloc[:, 1:-2].apply(lambda x: (x / x.sum()) * 1e6, axis=0)
df_normalized[[ 'ENSG', 'HGNC']] = df[['ENSG', 'HGNC']]
print("\nNormalized Data:\n")
print(df_normalized.head())



Normalized Data:

   Sample_1  Sample_2  Sample_3  Sample_4  Sample_5  Sample_6  Sample_7  \
0  0.000000  0.000000  0.000000  1.930194   0.00000  0.661822  0.000000   
1  0.224773  0.474113  0.000000  1.286796   0.71248  0.000000  0.184007   
2  0.224773  0.000000  0.216609  0.000000   0.00000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  1.930194   0.71248  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000   0.00000  0.000000  0.000000   

   Sample_8  Sample_9  Sample_10  ...  Sample_66  Sample_67  Sample_68  \
0       0.0       0.0        0.0  ...   0.000000        0.0        0.0   
1       0.0       0.0        0.0  ...   0.269096        0.0        0.0   
2       0.0       0.0        0.0  ...   0.067274        0.0        0.0   
3       0.0       0.0        0.0  ...   0.000000        0.0        0.0   
4       0.0       0.0        0.0  ...   0.000000        0.0        0.0   

   Sample_69  Sample_70  Sample_71  Sample_72  Sample_73             ENSG  \
0   0.00

In [87]:
# Step 5: Filtering the Data
# Filter out rows where the mean value across numeric samples is less than 1
numeric_columns = df_normalized.select_dtypes(include=[float]).columns
mean_values = df_normalized[numeric_columns].mean(axis=1)
df_filtered = df_normalized[mean_values > 1]
print("\nFiltered Data:\n")
print(df_filtered.head())


Filtered Data:

      Sample_1    Sample_2    Sample_3    Sample_4   Sample_5    Sample_6  \
19    4.270685    5.689356    4.332186    5.790582   1.424960    6.618221   
36    3.821139    5.215243    4.982014    5.790582   2.849920    0.661822   
37  140.483059  149.108525  130.832016  160.206093  70.535529   70.814968   
39  220.951756  290.631243  292.639161  178.221236  99.034732  120.451627   
41    3.146821    2.133508    3.465749    1.930194   3.562400    4.632755   

     Sample_7   Sample_8   Sample_9  Sample_10  ...  Sample_66  Sample_67  \
19   8.648318   6.212555   6.984612   7.365940  ...   1.681848   1.214139   
36   2.760101   2.070852   4.108595   2.946376  ...   0.538191   0.520345   
37  95.867521  91.376327  88.745654  76.114712  ...  64.381147  25.843814   
39  84.643109  78.174648  76.009009  69.730898  ...  79.517780  42.321413   
41   4.048149   1.553139   1.232579   0.982125  ...   0.605465   0.520345   

    Sample_68  Sample_69   Sample_70   Sample_71  Sample_

In [90]:
# Save the filtered and normalized DataFrame to a CSV file
df_filtered.to_csv('../data/filtered_normalized_gene_expression_data.csv', index=False)